# LAB-04 REGRESIÓN LOGÍSTICA MULTICLASE CON REGULARIZACIÓN Y SIN REGULARIZACIÓN 

### Nombre: Gonzales Suyo Franz Reinaldo

### Carrera: Ing. de Sistemas

### C.U. 35-5335

# Implementación del modelo de regresión logística multiclase

En este ejercicio se implementa regresion logistica multiclase y se aplica a dos diferentes datasets.

Nuestro objetuvo es predecir el estado de un hombre de estado civil, donde nuestro objetivo es predecir si el adulto tienen varios estados como: casado, soltero, viudo, sin-familia, etc.
En el siguiente dataset `adult.data.csv` se encuentran todos los datos sobre el equipo realiza en el proceso del partido.

Link del Dataset: https://data.world/uci/adult

Link del Repositorio de GitHub LAB-04: https://github.com/Gonzales-Franz-Reinaldo/SIS420-AI/tree/main/Laboratorios/LAB-04

In [2]:
# se utiliza para el manejo de rutas y directorios.
import os

# Calculo cientifico y vectorial para python
import numpy as np

# Librerias para graficar
import matplotlib.pyplot as plt

import pandas as pd

# Modulo de optimización de scipy
from scipy import optimize

#Para separa el Dataset 20% y 80% para diferentes pruebas
from sklearn.model_selection import train_test_split

# para aumentar datos en un dataset
from collections import Counter
from imblearn.over_sampling import SMOTE

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

# Informacióndel Dataset

Especificar cuales son sus  variables de entrada  (X): 

Variables de entrada son: 

X1:age: continuous.

X2:workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

X3: fnlwgt: continuous.

X4: education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

X5: education-num: continuous.

X6: marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

X7: occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

X8: relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

X9: race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

X10: sex: Female, Male.

X11: capital-gain: continuous.

X12: capital-loss: continuous.

X13: hours-per-week: continuous.

X14: native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

X15: class: >50K, <=50K

Y Label(Predicha):  Columna X6 "marital-status"


In [3]:
# Cargar los datos
data = pd.read_csv("./adult.data.csv", delimiter=",")

# Configurar Pandas para que no corte la visualización
pd.set_option('display.max_rows', 50)  # Mostrar todas las filas (60 -> None)
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas (20 -> None)

# Mostramos los datos del datset
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


age,workclass, fnlwgt, education, education-num, marital-status, occupation, relationship, race, sex,capital-gain, capital-loss, hours-per-week, native-country, class

# PREPARACIÓN DEL DATASET

In [4]:
#Leemos el tipo de datos que tiene el dataset y vemos los datos faltantes
print('INFORMACION DE TIPO DE DATOS')
data.info()
print('\nDATOS VACIOS')
print(pd.isnull(data).sum())

INFORMACION DE TIPO DE DATOS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  class           32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB

DATOS VACIOS
age               0
workclass         0
fnlwgt    

In [5]:
# Convertimos los valores de los estados o clases del adulto a representar con numeros, los siguientes 7 estados 
estado_matrimonio = {
    ' Married-civ-spouse': 0,
    ' Divorced': 1,
    ' Never-married': 2,
    ' Separated': 3,
    ' Widowed': 4,
    ' Married-spouse-absent': 5,
    ' Married-AF-spouse': 6
}


# Reemplazar los valores en la columna "marital-status" con los valores numéricos
data["marital-status"] = data["marital-status"].replace(estado_matrimonio)

# Imprimir datos
data

C:\Users\gonza\AppData\Local\Temp\ipykernel_15172\3157297133.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data["marital-status"] = data["marital-status"].replace(estado_matrimonio)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,2,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,0,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,1,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,0,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,0,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,0,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,0,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,4,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,2,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [6]:
#labelEncoder sirve para convertir datos categoricos a numericos para poder hacer mejores calculos
from sklearn.preprocessing import LabelEncoder

#variables categoricas, donde convertimos a int64
columnas_categoricas = data.select_dtypes(include=['object']).columns

#Procesamiento de frases o palabras a valores numericos para su procesamiento
for columna in columnas_categoricas:
    le = LabelEncoder()
    data[columna] = le.fit_transform(data[columna])

In [7]:
#Verificamos todos los datos del Dataset

print("\nTIPOS DE DATOS")
data.info()
print("\nDATOS VACIOS")
print(pd.isnull(data).sum())


TIPOS DE DATOS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             32561 non-null  int64
 1   workclass       32561 non-null  int32
 2   fnlwgt          32561 non-null  int64
 3   education       32561 non-null  int32
 4   education-num   32561 non-null  int64
 5   marital-status  32561 non-null  int64
 6   occupation      32561 non-null  int32
 7   relationship    32561 non-null  int32
 8   race            32561 non-null  int32
 9   sex             32561 non-null  int32
 10  capital-gain    32561 non-null  int64
 11  capital-loss    32561 non-null  int64
 12  hours-per-week  32561 non-null  int64
 13  native-country  32561 non-null  int32
 14  class           32561 non-null  int32
dtypes: int32(8), int64(7)
memory usage: 2.7 MB

DATOS VACIOS
age               0
workclass         0
fnlwgt            0
education         0
ed

In [8]:
#Movimiento de columna al final del dataset y verificacion de datos
# En este caso movemos nuestra Y predicha al final de las columnas que en este caso es "4206.0.1"
indice_columna = data.columns.get_loc('marital-status')
columnas = list(data.columns)
columnas.pop(indice_columna)
columnas.append('marital-status')
data = data[columnas]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             32561 non-null  int64
 1   workclass       32561 non-null  int32
 2   fnlwgt          32561 non-null  int64
 3   education       32561 non-null  int32
 4   education-num   32561 non-null  int64
 5   occupation      32561 non-null  int32
 6   relationship    32561 non-null  int32
 7   race            32561 non-null  int32
 8   sex             32561 non-null  int32
 9   capital-gain    32561 non-null  int64
 10  capital-loss    32561 non-null  int64
 11  hours-per-week  32561 non-null  int64
 12  native-country  32561 non-null  int32
 13  class           32561 non-null  int32
 14  marital-status  32561 non-null  int64
dtypes: int32(8), int64(7)
memory usage: 2.7 MB


### Precedimiento para dividir las clases, cada clase debe tener el 80% para e entrenamiento y el 20% para la prueba

In [9]:
# Como nuestro columna de Y es la martial-status, contamos los numeros de ocurriencias de cada clase
class_count = data['marital-status'].value_counts()

# Mostramos las cantidad de clases que existen en el dataset
class_count

marital-status
0    14976
2    10683
1     4443
3     1025
4      993
5      418
6       23
Name: count, dtype: int64

Como observamos en el dataset tenemos 7 clases que van del 0 al 6 pero esto nos puede complicar en el futuro, por lo que vamos a eliminar las clases 5 y 6.
El entrenamiento seria un poco regular, en este caso solo tendremos 5 clases del 0 al 4

In [10]:

# Como nuestro columna de Y es la marital-status, contamos los numeros de ocurrencias de cada clase
class_count = data['marital-status'].value_counts()

# Mostramos la cantidad de clases que existen en el dataset original
print("Antes de eliminar:")
print(class_count)

# Guardamos las clases que deseamos eliminar
classes_to_remove = [5, 6]

# Filtramos el DataFrame original para eliminar las clases seleccionadas
df = data[~data['marital-status'].isin(classes_to_remove)]


Antes de eliminar:
marital-status
0    14976
2    10683
1     4443
3     1025
4      993
5      418
6       23
Name: count, dtype: int64


In [11]:
# Mostramos el datset preparado
df

,age,workclass,fnlwgt,education,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,marital-status
0,39,7,77516,9,13,1,1,4,1,2174,0,40,39,0,2
1,50,6,83311,9,13,4,0,4,1,0,0,13,39,0,0
2,38,4,215646,11,9,6,1,4,1,0,0,40,39,0,1
3,53,4,234721,1,7,6,0,2,1,0,0,40,39,0,0
4,28,4,338409,9,13,10,5,2,0,0,0,40,5,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,4,257302,7,12,13,5,4,0,0,0,38,39,0,0
32557,40,4,154374,11,9,7,0,4,1,0,0,40,39,1,0
32558,58,4,151910,11,9,1,4,4,0,0,0,40,39,0,4
32559,22,4,201490,11,9,1,3,4,1,0,0,20,39,0,2


In [12]:
# Como nuestro columna de Y es la martial-status, contamos los numeros de ocurriencias de cada clase
class_count = df['marital-status'].value_counts()

# Mostramos las cantidad de clases que existen en el dataset
class_count

marital-status
0    14976
2    10683
1     4443
3     1025
4      993
Name: count, dtype: int64

#### Precedimiento para dividir las las clases, cada clase debe tener el 80% para e entrenamiento y el 20% para la prueba

In [13]:
# Calculamos el 80% y el 20% indices para cada clase 

# Calculamos el 80% y el 20% de índices para cada clase 
class_indices_train = {c: int(count * 0.8) for c, count in class_count.items()}
class_indices_test = {c: int(count * 0.2) for c, count in class_count.items()}

# Dividimos los datos en conjunto de entrenamiento y prueba
X_train, y_train = [], []
X_test, y_test = [], []

for class_name, indices_train in class_indices_train.items():
    indices_test = class_indices_test[class_name]  # Obtenemos el número de índices de prueba correspondientes a esta clase
    
    # Seleccionamos los índices para el conjunto de entrenamiento
    indices_train_data = data[data['marital-status'] == class_name].index[:indices_train]
    X_train.extend(data.loc[indices_train_data].drop('marital-status', axis=1).values.tolist())
    y_train.extend([class_name] * len(indices_train_data))
    
    # Seleccionamos los índices para el conjunto de prueba
    indices_test_data = data[data['marital-status'] == class_name].index[indices_train:indices_train + indices_test]
    X_test.extend(data.loc[indices_test_data].drop('marital-status', axis=1).values.tolist())
    y_test.extend([class_name] * len(indices_test_data))

# Convertimos el conjunto de entrenamiento y de prueba a DataFrames
X_train = pd.DataFrame(X_train, columns=data.columns[:-1])
X_test = pd.DataFrame(X_test, columns=data.columns[:-1])

y_train = pd.DataFrame(y_train, columns=['marital-status'])
y_test = pd.DataFrame(y_test, columns=['marital-status'])


In [17]:
import random

# Mezclar los índices de los datos
indices = list(range(len(X_train)))
random.shuffle(indices)

# Reordenar los datos de entrenamiento según los índices mezclados
X_train = X_train.iloc[indices].reset_index(drop=True)
y_train = y_train.iloc[indices].reset_index(drop=True)


In [18]:
# Mezclar indices para los datos de X_test y y_test
# Mezclar los índices de los datos
indices = list(range(len(X_test)))
random.shuffle(indices)

# Reordenar los datos de entrenamiento según los índices mezclados
X_test = X_test.iloc[indices].reset_index(drop=True)
y_test = y_test.iloc[indices].reset_index(drop=True)

In [19]:
# Verificamos que sí esta cumpliendo la cantidad de datos del 80% de cada clase para el entrenamiento de X_train y y_train 
y_train_class_count = y_train['marital-status'].value_counts()
y_train_class_count

marital-status
0    11980
2     8546
1     3554
3      820
4      794
Name: count, dtype: int64

In [20]:
# Mostramos los datos de X_train para el entrenamiento
X_train

,age,workclass,fnlwgt,education,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,20,4,170800,15,10,5,3,4,0,0,0,40,39,0
1,39,4,111268,9,13,1,3,4,0,0,0,40,39,0
2,17,4,347322,0,6,12,3,4,0,0,0,20,39,0
3,24,4,103064,9,13,10,1,4,0,0,0,40,39,0
4,46,4,376789,11,9,8,1,4,1,0,0,15,39,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25689,24,4,185821,15,10,12,3,4,0,0,0,40,39,0
25690,26,4,193945,7,12,13,1,4,1,0,0,45,39,0
25691,44,4,79531,8,11,3,0,4,1,0,0,40,39,1
25692,25,4,28473,8,11,3,0,4,1,0,0,40,39,0


Una vez que tengamos los datos preprocesados, guardamos una copia

In [47]:
train_data = pd.concat([X_train, y_train], axis=1)
test_data = pd.concat([X_test, y_test], axis=1)

In [50]:
train_data.to_csv('../LAB-04/adult_train_data.csv', index=False)
test_data.to_csv('../LAB-04/adult_test_data.csv', index=False)

# 1.1 CLASIFICACIÓN MULTICLASE CON REGULARIZACIÓN

Este dataset tiene una conjunto de datos donde reconoce el tipo de estado civil de un adulto que puede estar
en diferentes tipos de estados o clases como: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

Se inicia el ejercicio cargando los datos 

In [51]:
df_train = pd.read_csv('./adult_train_data.csv')
df_test = pd.read_csv('./adult_test_data.csv')

# Datos de entrenamiento del 80% 
X_train = df_train.drop('marital-status', axis=1)
y_train = df_train['marital-status']


# Datos del prueba del 20 % 
X_test = df_test.drop('marital-status', axis=1)
y_test = df_test['marital-status']

In [52]:
# Inicializamos la entrada mas el x0 en X_train
input_layer_size = 15

num_labels = 7

# y_train[y_train == 0] = 1;

m = y_train.size 

In [53]:
# Datos de X_train para el entrenamiento 
print("Datos de X_train")
print(X_train)

print('=' * 100)

print("Datos de y_train")
print(y_train)

# Mostramos la cantidad de ejemplos que se utilizaran para el entrenamiento
print('=' * 100)
print("Cantidad de ejemplos del 80% para el entrenamiento es de: {:.0f}".format(len(X_train)))
print("Cantidad de ejemplos del 20% para la prueba es de: {:.0f}".format(len(X_test)))


Datos de X_train
       age  workclass  fnlwgt  education  education-num  occupation  \
0       20          4  170800         15             10           5   
1       39          4  111268          9             13           1   
2       17          4  347322          0              6          12   
3       24          4  103064          9             13          10   
4       46          4  376789         11              9           8   
...    ...        ...     ...        ...            ...         ...   
25689   24          4  185821         15             10          12   
25690   26          4  193945          7             12          13   
25691   44          4   79531          8             11           3   
25692   25          4   28473          8             11           3   
25693   36          4  139743         15             10          10   

       relationship  race  sex  capital-gain  capital-loss  hours-per-week  \
0                 3     4    0             0        

## Definimos la funcion de normalización 
Para normalizar los datos de X_train, esto para que el descenso por el gradiente sea mas rápido y no sea tan dependiente de los valores iniciales.

In [54]:
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [55]:
# Llamamos a la function de featureNormalize para normalizar los datos de entrenamiento
X_norm, mu, sigma = featureNormalize(X_train)

# Mostramos los datos de X_train normalizados
X_norm

,age,workclass,fnlwgt,education,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,-1.358529,0.088814,-0.179037,1.214815,-0.036478,-0.373208,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
1,0.029830,0.088814,-0.746179,-0.338802,1.139008,-1.317190,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
2,-1.577744,0.088814,1.502630,-2.669226,-1.603792,1.278761,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-1.661160,0.283824,-0.565754
3,-1.066243,0.088814,-0.824336,-0.338802,1.139008,0.806770,-0.270524,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
4,0.541330,0.088814,1.783353,0.179070,-0.428306,0.334779,-0.270524,0.388385,0.698106,-0.146356,-0.216102,-2.067847,0.283824,-0.565754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25689,-1.066243,0.088814,-0.035937,1.214815,-0.036478,1.278761,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
25690,-0.920100,0.088814,0.041458,-0.856674,0.747180,1.514756,-0.270524,0.388385,0.698106,-0.146356,-0.216102,0.372273,0.283824,-0.565754
25691,0.395187,0.088814,-1.048528,-0.597738,0.355351,-0.845199,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-0.034413,0.283824,1.767551
25692,-0.993171,0.088814,-1.534941,-0.597738,0.355351,-0.845199,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-0.034413,0.283824,-0.565754


## 1.2 Visualización de los datos

In [56]:
# Selecciona aleatoriamente  100 puntos de datos para mostrar
rand_indices = np.random.choice(m, 100, replace=False)
sel = X_train.iloc[rand_indices, :]

## 1.3 Vectorización de regresión logística

La regresin logistica de multivariable se utilizará en multiple modelos para construir un clasificador
donde tendremos 7 clases, donde deberá entrenar 7 clasificadores de regresión logística separados.  

In [57]:
# definimos la función de sigmoide
def sigmoid(z):
    # Calcula la sigmoide de z 
    return 1.0 / (1.0 + np.exp(-z))


### 1.3.1 Vectorización de la función de costo

### 1.3.2 Vectorización del gradiente

Calculamos el costo de usar theta como parametro para la regresión logística regularizada y el gradiente del costo 
a los parámetros

In [58]:
def costoFunction(theta, X, y, lambda_):
    # lambda es un parametro de regularización
    m = y.size
    
    # convierte la etiquetas a valores enteros si son boleanos
    if y.dtype == bool:
        y = y.astype(int)
    
    J = 0
    
    #* grad es un vecto de la forma (shape, n) que es el gradiente de la funcion del costo
    # con respecto a theta, en los valores actuales de theta
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    temp = theta
    temp[0] = 0
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))
    
    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp
    
    return J, grad


### 1.3.3 Vectorización regularizada de la regresión logística

Una vez implementado la vectorización para la regresión logística, agregamos la regularización 
a la función del costo.

### 1.4 Clasificación One-vs-all

En el ejercicio implementaremos la clasificación de una contra todos mediante el entrenamiento de multiples 
clasificadores de regresión logística regularizados, para cada clase que tenemos 7 clases
Debemos entrenar el clasificador para todas las etiquetas del 0 al 6

In [59]:
# Definimos la funcion de oneVsAll
def oneVsAll(X, y, num_labels, lambda_):
    #* algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))
    
    #? Agregar unos a la matriz X 
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(costoFunction, 
                                initial_theta,
                                (X, (y == c), lambda_),
                                jac=True,
                                method='CG',
                                options=options)
        all_theta[c] = res.x
        
    return all_theta

Entrenamiento del modelo

In [60]:
# Llamamos a la función 
lambda_ = 0.1
all_theta = oneVsAll(X_norm, y_train.values.ravel(), num_labels, lambda_)
print(all_theta.shape)

#? y_train.values.ravel() se aplica esto porque es un dataframe

(7, 15)


In [61]:
# Mostramos todas las thetas
print(all_theta)

[[-1.49919852e-01  4.41009686e-01  8.66200748e-02 -4.71248393e-02
   3.61756066e-02 -1.88746937e-01 -4.36983092e-02 -8.30552689e-01
   5.85108388e-02  5.42153076e-01 -7.48911038e-02 -8.76209691e-03
   7.76267735e-02 -7.18151174e-02  1.04344227e+00]
 [-2.19894896e+00  6.01915598e-01  3.31397533e-02  1.48813197e-02
   7.78018627e-02  7.03491796e-02 -1.17171985e-02  4.71778484e-01
   9.70615877e-02 -4.13016878e-01  3.22778329e-02 -1.38205483e-02
   3.50076322e-01  1.16978799e-01 -5.84542607e-01]
 [-1.32195600e+00 -1.63848100e+00 -5.30683407e-02 -4.70010784e-04
  -8.35307584e-02  3.47580123e-01  3.86968435e-02  3.50143336e-01
  -9.53884842e-02 -4.04808457e-02  6.02682219e-02  8.47906830e-03
  -2.00867813e-01 -2.51911743e-03 -7.60818361e-01]
 [-3.84793441e+00  2.18897801e-01  2.38232245e-02  1.75600734e-01
   1.60928442e-02 -2.25465253e-01  4.19999800e-02  4.71855896e-01
  -1.77554190e-01 -2.69602715e-01 -5.43464532e-03  1.85085049e-02
   1.42992666e-01 -3.02576392e-02 -5.30970092e-01]
 [-5

## 1.4.1 Predición One-vs-All

Una vez entrenado el clasificador de one-vs-all, se puede usarlo para predecir el estado o clase que tiene el adulto en este caso va del 0 al 6 donde se representan:
    Married-civ-spouse: 0,
    Divorced: 1,
    Never-married: 2,
    Separated: 3,
    Widowed: 4,
    Married-spouse-absent: 5,
    Married-AF-spouse: 6

Entonces para cada entrada, debe calcular la "probabilidad" de que pertenezca a cada clase utilizando los clasificadores de regresión logística entrenados. La función de predicción one-vs-all seleccionará la clase para la cual el clasificador de regresión logística correspondiente genera la probabilidad más alta y devolverá la etiqueta de clase (0, 1, 2,..., 6) como la predicción para el ejemplo de entrada.

In [62]:
# Definimos la funcion de predicción de oneVsAll
def predctOneVSAll(all_theta, X):
    
    m = X.shape[0]
    num_labels = all_theta.shape[0]  # 7 labels
    
    p = np.zeros(m)
    
    # Agregamos unos a la matriz X 
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)
    
    return p

Esta funcion nos devolverá un vector de predicciones para cada ejemplo en la matrx de X_norm
Donde all_theta es una matriz donde la enésima fila es un vector theta de regresión logística que entrenó para 
todas las calses en este caso las 7 clases

Una vez que haya terminado, llamamos a la función predictOneVsAll usando el valor aprendido de theta, donde debe tener 
una buena precisión para clasificar en conjunto de datos de entrenamiento

In [70]:
# Hacemos la pruebe de prediccion de los clasificadores
print(X_norm.shape)

pred = predctOneVSAll(all_theta, X_norm)

print('Precisión del conjunto de entrenamiento  es: {:.2f}%'.format(np.mean(pred == y_train) * 100))

XPrueba = X_norm.iloc[10:500, :].copy()
print(XPrueba.shape)

XPrueba = np.concatenate([np.ones((490, 1)), XPrueba], axis=1)
print(XPrueba.shape)

p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis=1)

print(p)
print('=' * 100)

# print(y_train.iloc[10:200].values)
# y_train_array = y_train.iloc[10:500]['marital-status'].values
y_train_array = y_train[10:500].values

print(y_train_array)


(25694, 14)
Precisión del conjunto de entrenamiento  es: 71.99%
(490, 14)
(490, 15)
[2 2 0 0 1 0 2 2 2 2 2 0 2 0 2 0 2 0 2 0 2 0 2 1 0 1 0 0 2 0 0 2 0 1 0 0 2
 2 0 0 0 1 1 2 2 0 0 0 0 1 2 0 0 2 0 0 0 4 0 0 2 0 2 1 2 0 0 0 0 0 0 0 0 0
 2 2 0 4 0 2 2 0 0 0 0 0 0 2 2 0 2 2 2 0 2 2 1 0 0 0 0 2 2 2 2 0 2 0 0 0 0
 0 2 0 0 2 0 0 0 2 0 2 2 2 0 0 2 2 2 0 0 0 1 0 2 2 0 0 2 2 0 0 0 0 2 0 1 2
 0 0 2 0 1 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 2 0 0 2 2 0 2 0 2 2 0 1
 2 0 2 1 1 0 2 2 0 0 0 2 0 0 2 0 2 1 2 0 0 2 4 2 2 0 0 0 0 0 2 2 2 0 2 2 2
 2 0 2 1 2 2 0 2 2 1 2 2 0 0 0 0 0 2 0 2 0 2 2 2 2 2 2 2 0 0 0 2 0 2 2 2 2
 0 0 2 2 2 2 2 0 0 2 0 2 0 2 0 2 2 2 2 0 0 0 0 0 2 0 0 2 1 0 2 1 0 2 0 0 2
 1 1 0 0 2 2 2 1 0 0 0 0 0 2 2 0 0 0 0 0 0 2 0 0 0 0 2 0 1 2 1 2 2 0 2 0 2
 2 0 0 2 2 0 0 0 2 2 0 2 0 2 0 2 2 0 0 2 2 1 0 0 0 2 0 0 0 0 0 2 0 2 2 0 2
 2 0 0 1 0 0 2 2 0 2 1 2 0 2 0 2 2 2 4 2 1 0 0 2 0 2 0 0 0 0 0 2 0 1 2 0 2
 2 2 2 2 0 0 2 2 2 0 0 1 0 0 0 0 2 2 0 0 0 0 0 2 0 2 0 1 0 0 2 0 0 0 1 0 2
 2 1 0 2 0 0 0 0

### Otras predicciones con otro tipo de valores

In [73]:
# Hacemos la pruebe de prediccion de los clasificadores
print(X_norm.shape)

pred = predctOneVSAll(all_theta, X_norm)

print('Precisión del conjunto de entrenamiento  es: {:.2f}%'.format(np.mean(pred == y_train.values) * 100))

XPrueba = X_norm.iloc[20:700, :].copy()
print(XPrueba.shape)

XPrueba = np.concatenate([np.ones((680, 1)), XPrueba], axis=1)
print(XPrueba.shape)

p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis=1)

print(p)

# print(y_train.iloc[10:200].values)
y_train_array = y_train[20:700].values

print(y_train_array)


(25694, 14)
Precisión del conjunto de entrenamiento  es: 71.99%
(680, 14)
(680, 15)
[2 0 2 0 2 0 2 0 2 0 2 0 2 1 0 1 0 0 2 0 0 2 0 1 0 0 2 2 0 0 0 1 1 2 2 0 0
 0 0 1 2 0 0 2 0 0 0 4 0 0 2 0 2 1 2 0 0 0 0 0 0 0 0 0 2 2 0 4 0 2 2 0 0 0
 0 0 0 2 2 0 2 2 2 0 2 2 1 0 0 0 0 2 2 2 2 0 2 0 0 0 0 0 2 0 0 2 0 0 0 2 0
 2 2 2 0 0 2 2 2 0 0 0 1 0 2 2 0 0 2 2 0 0 0 0 2 0 1 2 0 0 2 0 1 0 2 0 0 0
 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 2 0 0 2 2 0 2 0 2 2 0 1 2 0 2 1 1 0 2 2 0 0
 0 2 0 0 2 0 2 1 2 0 0 2 4 2 2 0 0 0 0 0 2 2 2 0 2 2 2 2 0 2 1 2 2 0 2 2 1
 2 2 0 0 0 0 0 2 0 2 0 2 2 2 2 2 2 2 0 0 0 2 0 2 2 2 2 0 0 2 2 2 2 2 0 0 2
 0 2 0 2 0 2 2 2 2 0 0 0 0 0 2 0 0 2 1 0 2 1 0 2 0 0 2 1 1 0 0 2 2 2 1 0 0
 0 0 0 2 2 0 0 0 0 0 0 2 0 0 0 0 2 0 1 2 1 2 2 0 2 0 2 2 0 0 2 2 0 0 0 2 2
 0 2 0 2 0 2 2 0 0 2 2 1 0 0 0 2 0 0 0 0 0 2 0 2 2 0 2 2 0 0 1 0 0 2 2 0 2
 1 2 0 2 0 2 2 2 4 2 1 0 0 2 0 2 0 0 0 0 0 2 0 1 2 0 2 2 2 2 2 0 0 2 2 2 0
 0 1 0 0 0 0 2 2 0 0 0 0 0 2 0 2 0 1 0 0 2 0 0 0 1 0 2 2 1 0 2 0 0 0 0 2 1
 2 2 0 0 0 1 0 2

### 1.4.2 Validaciones y Prueba


Para las validaciones correspondientes utilizamos datos del entrenamiento dodne aplicamos la forma que 80%
y el 20% para la fase de prueba.''

In [74]:
# Aplicaremos datos del dataframe X_test que son el 20% para la prueba 

# Normalizamos el dataset de X_test
X_test_norm = (X_test - mu) / sigma
m_test = len(X_test)

Normalizamos con la columna de (1) al dataframe de X_test

In [75]:
# Mostramos los X_test normalizados
X_test_norm

,age,workclass,fnlwgt,education,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,0.760545,-1.973228,0.414084,-0.597738,0.355351,-0.845199,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
1,2.587333,1.463509,-0.766824,1.214815,-0.036478,-0.609204,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-2.962557,0.283824,-0.565754
2,-0.554742,-1.285880,0.093121,-0.338802,1.139008,1.042765,-0.892912,-1.993562,0.698106,-0.146356,-0.216102,0.778960,0.283824,1.767551
3,-0.043242,2.150856,2.603057,1.214815,-0.036478,-0.609204,2.219030,-1.993562,-1.432448,-0.146356,-0.216102,-0.034413,0.153783,-0.565754
4,0.102901,0.088814,0.126502,0.179070,-0.428306,1.278761,2.219030,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,1.767551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6417,0.249044,0.088814,-0.680026,-1.115610,-1.995620,1.750752,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
6418,-0.043242,-2.660575,-0.782276,-1.115610,-1.995620,-1.553186,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-2.311859,0.283824,-0.565754
6419,1.564332,0.088814,-0.444622,0.179070,-0.428306,-0.845199,-0.892912,0.388385,0.698106,-0.146356,-0.216102,0.778960,0.283824,-0.565754
6420,0.833617,0.088814,-0.483396,1.214815,-0.036478,-0.845199,-0.270524,0.388385,0.698106,0.471444,-0.216102,-1.498485,0.283824,-0.565754


### Realizamos el calculo de la Y predicha con los datros de prueba de X_test

In [76]:
# Declaramos un vector donde se calcularan las y_predicha
y_predicha = []

# Calculamos la Y predicha de cada fila de X_test_ready
for i in range(len(X_test_norm)):
    y_predicha.append(predctOneVSAll(all_theta, X_test_norm.iloc[i, :].values.reshape(1, -1)))
    
# Mostramos la Y predicha
y_predicha = np.array(y_predicha).flatten()
y_predicha

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [77]:
# Calculamos su precisión
precision = np.mean(y_predicha == y_test.values) * 100
print("La precisión del conjunto de prueba es de: {:.2f}%".format(precision))

La precisión del conjunto de prueba es de: 71.61%


### Predicción con los datos de prueba X_test para las predicciones

In [78]:
pred_test = predctOneVSAll(all_theta, X_test_norm)

# Mostramos los resultados predecidos 
pred_test

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [79]:
# Mostramos una tabla con los datos de prueba y los resultados predecidos

y_pred_test = pd.DataFrame(pred_test, columns=['marital-status-(pred)'])

tabla_predicciones = pd.concat([X_test, y_test, y_pred_test], axis=1)

# Mostramos la tabla de resultados 
tabla_predicciones.head(20)

,age,workclass,fnlwgt,education,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,marital-status,marital-status-(pred)
0,49,1,233059,8,11,3,0,4,1,0,0,40,39,0,0,0
1,74,6,109101,15,10,4,0,4,1,0,0,4,39,0,0,0
2,31,2,199368,9,13,11,0,2,1,0,0,50,39,1,0,0
3,38,7,462832,15,10,4,5,2,0,0,0,40,38,0,0,2
4,40,4,202872,11,9,12,5,4,0,0,0,40,39,1,0,0
5,60,0,134152,6,5,0,1,2,1,0,0,35,39,0,1,0
6,43,4,221172,9,13,4,1,4,1,0,0,35,39,0,1,0
7,27,4,59068,11,9,6,0,4,1,0,0,40,39,0,0,0
8,53,5,220786,15,10,12,1,4,0,0,0,60,39,0,4,1
9,45,4,156117,8,11,10,1,4,0,0,0,32,39,0,1,0


### Calculo de la precisión que tiene el modelo de la prueba

Calculamos con el metodo np.mean, la media(promedio) de los valores booleanos. Donde True se considera como 1 y False como 0 en operaciones aritméticas, la media resultante será la proporción de los elementos iguales en pred_test e y_test

In [80]:
# Evaluación del rendimiento de datos de prueba 
accuracy_test = np.mean(pred_test == y_test.values) * 100

print("La precisión del conjunto de prueba es de: {:.2f}%".format(accuracy_test))

La precisión del conjunto de prueba es de: 71.61%


## Conclusión de la práctica

Segun los resultados tanto de predicciónes y la precisión del entrenamiento del modelo es de 71.61% y podemos ver que
está por inferior del 50% por lo que el modelo está regular, en este caso que es el objetivo de predecir si un hombre 
adulto se identifica a una clase de 0 al 4, es necesario mejorar el modelo para que tenga una mejor precisión.

Por lo que la probabilidad de que un hombre adulto se identifique a una clase de 0 al 4 es de 71.61% y que el mismo es mayor al 50%.

### =============================================================================================================================

# REGRESIÓN LOGÍATICA MULTICLASE SIN REGURLARIZACIÓN

In [1]:
# se utiliza para el manejo de rutas y directorios.
import os

# Calculo cientifico y vectorial para python
import numpy as np

# Librerias para graficar
import matplotlib.pyplot as plt

import pandas as pd

# Modulo de optimización de scipy
from scipy import optimize

#Para separa el Dataset 20% y 80% para diferentes pruebas
from sklearn.model_selection import train_test_split

# para aumentar datos en un dataset
from collections import Counter
from imblearn.over_sampling import SMOTE

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

In [42]:
df_train = pd.read_csv('./adult_train_data.csv')
df_test = pd.read_csv('./adult_test_data.csv')

# Datos de entrenamiento del 80% 
X_train = df_train.drop('marital-status', axis=1)
y_train = df_train['marital-status']


# Datos del prueba del 20 % 
X_test = df_test.drop('marital-status', axis=1)
y_test = df_test['marital-status']

In [43]:
# Inicializamos la entrada mas el x0 en X_train
input_layer_size = 15

num_labels = 7

# y_train[y_train == 0] = 1;

m = y_train.size 

### 1 Definimos la funcion de normalización

In [44]:
# Definimos la funcion de normalizacion
def  featureNormalize(X):
    X_norm = X.copy()
    mu = np.zeros(X.shape[1])
    sigma = np.zeros(X.shape[1])

    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)
    X_norm = (X - mu) / sigma

    return X_norm, mu, sigma

In [45]:
# Llamamos a la function de featureNormalize para normalizar los datos de entrenamiento
X_norm, mu, sigma = featureNormalize(X_train)

# Mostramos los datos de X_train normalizados
X_norm

,age,workclass,fnlwgt,education,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,-1.358529,0.088814,-0.179037,1.214815,-0.036478,-0.373208,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
1,0.029830,0.088814,-0.746179,-0.338802,1.139008,-1.317190,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
2,-1.577744,0.088814,1.502630,-2.669226,-1.603792,1.278761,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-1.661160,0.283824,-0.565754
3,-1.066243,0.088814,-0.824336,-0.338802,1.139008,0.806770,-0.270524,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
4,0.541330,0.088814,1.783353,0.179070,-0.428306,0.334779,-0.270524,0.388385,0.698106,-0.146356,-0.216102,-2.067847,0.283824,-0.565754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25689,-1.066243,0.088814,-0.035937,1.214815,-0.036478,1.278761,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
25690,-0.920100,0.088814,0.041458,-0.856674,0.747180,1.514756,-0.270524,0.388385,0.698106,-0.146356,-0.216102,0.372273,0.283824,-0.565754
25691,0.395187,0.088814,-1.048528,-0.597738,0.355351,-0.845199,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-0.034413,0.283824,1.767551
25692,-0.993171,0.088814,-1.534941,-0.597738,0.355351,-0.845199,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-0.034413,0.283824,-0.565754


## 1.2 Vectorización de regresión logística

In [46]:
# definimos la función de sigmoide
def sigmoid(z):
    # Calcula la sigmoide de z 
    return 1.0 / (1.0 + np.exp(-z))

## 1.2.1 Vectorización de la función de costo

## 1.2.2 Vectorización del gradiente

In [47]:
# 1.2.1 Vectorización de la función de costo
# 1.2.2 Vectorización del gradiente

def costoFunction(theta, X, y):
    # lambda es un parametro de regularización
    m = y.size
    
    J = 0
    
    #* grad es un vecto de la forma (shape, n) que es el gradiente de la funcion del costo
    # con respecto a theta, en los valores actuales de theta
    grad = np.zeros(theta.shape)
    
    h = sigmoid(X.dot(theta.T))
    
    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h)))
    
    grad = (1 / m) * (h - y).dot(X)
    
    return J, grad

## 1.3 Clasificación One-vs-all

In [48]:
# Definimos la funcion de oneVsAll
def oneVsAll(X, y, num_labels):
    #* algunas variables utiles
    m, n = X.shape
    
    all_theta = np.zeros((num_labels, n + 1))
    
    #? Agregar unos a la matriz X 
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    
    for c in np.arange(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(costoFunction, 
                                initial_theta,
                                (X, (y == c)),
                                jac=True,
                                method='CG',
                                options=options)
        all_theta[c] = res.x
        
    return all_theta

In [49]:
# Llamamos a la función 
all_theta = oneVsAll(X_norm, y_train.values.ravel(), num_labels)
print(all_theta.shape)

#? y_train.values.ravel() se aplica esto porque es un dataframe

# Mostramos todas las thetas
print(all_theta)

(7, 15)
[[-1.49912019e-01  4.41014500e-01  8.66229843e-02 -4.71275385e-02
   3.61828875e-02 -1.88767681e-01 -4.37018746e-02 -8.30587271e-01
   5.85101200e-02  5.42162569e-01 -7.49058769e-02 -8.76795380e-03
   7.76200826e-02 -7.18178690e-02  1.04350792e+00]
 [-2.19898466e+00  6.01949953e-01  3.31409929e-02  1.48854589e-02
   7.78039209e-02  7.03612696e-02 -1.17161046e-02  4.71803801e-01
   9.70696657e-02 -4.13022649e-01  3.22876187e-02 -1.38179910e-02
   3.50100635e-01  1.16985934e-01 -5.84598674e-01]
 [-1.32202051e+00 -1.63859082e+00 -5.30675749e-02 -4.73036142e-04
  -8.35431081e-02  3.47618469e-01  3.86974162e-02  3.50153371e-01
  -9.53946475e-02 -4.04787097e-02  6.02840540e-02  8.48583150e-03
  -2.00865722e-01 -2.52009776e-03 -7.60871043e-01]
 [-3.84806172e+00  2.18944735e-01  2.38252327e-02  1.75621860e-01
   1.61056240e-02 -2.25479781e-01  4.20136624e-02  4.71920172e-01
  -1.77559233e-01 -2.69596599e-01 -5.39349916e-03  1.85276571e-02
   1.43030637e-01 -3.02557648e-02 -5.31188630e-

## 1.3.1 Predición One-vs-All

In [50]:
# Definimos la funcion de predicción de oneVsAll
def predctOneVSAll(all_theta, X):
    
    m = X.shape[0]
    num_labels = all_theta.shape[0]  # 7 labels
    
    p = np.zeros(m)
    
    # Agregamos unos a la matriz X 
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis = 1)
    
    return p

 Hacemos la pruebe de prediccion de los clasificadores

In [53]:
# Hacemos la pruebe de prediccion de los clasificadores
print(X_norm.shape)

pred_reg = predctOneVSAll(all_theta, X_norm)

print('Precisión del conjunto de entrenamiento  es: {:.2f}%'.format(np.mean(pred_reg == y_train) * 100))

XPrueba = X_norm.iloc[10:500, :].copy()
print(XPrueba.shape)

XPrueba = np.concatenate([np.ones((490, 1)), XPrueba], axis=1)
print(XPrueba.shape)

p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis=1)

# Mostramos los valores predichos
print(p)
print('=' * 100)

# Mostramos los valores de Y del 80% de los datos de entrenamiento
y_train_array = y_train[10:500].values

print(y_train_array)

(25694, 14)
Precisión del conjunto de entrenamiento  es: 71.99%
(490, 14)
(490, 15)
[2 2 0 0 1 0 2 2 2 2 2 0 2 0 2 0 2 0 2 0 2 0 2 1 0 1 0 0 2 0 0 2 0 1 0 0 2
 2 0 0 0 1 1 2 2 0 0 0 0 1 2 0 0 2 0 0 0 4 0 0 2 0 2 1 2 0 0 0 0 0 0 0 0 0
 2 2 0 4 0 2 2 0 0 0 0 0 0 2 2 0 2 2 2 0 2 2 1 0 0 0 0 2 2 2 2 0 2 0 0 0 0
 0 2 0 0 2 0 0 0 2 0 2 2 2 0 0 2 2 2 0 0 0 1 0 2 2 0 0 2 2 0 0 0 0 2 0 1 2
 0 0 2 0 1 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 2 0 0 2 2 0 2 0 2 2 0 1
 2 0 2 1 1 0 2 2 0 0 0 2 0 0 2 0 2 1 2 0 0 2 4 2 2 0 0 0 0 0 2 2 2 0 2 2 2
 2 0 2 1 2 2 0 2 2 1 2 2 0 0 0 0 0 2 0 2 0 2 2 2 2 2 2 2 0 0 0 2 0 2 2 2 2
 0 0 2 2 2 2 2 0 0 2 0 2 0 2 0 2 2 2 2 0 0 0 0 0 2 0 0 2 1 0 2 1 0 2 0 0 2
 1 1 0 0 2 2 2 1 0 0 0 0 0 2 2 0 0 0 0 0 0 2 0 0 0 0 2 0 1 2 1 2 2 0 2 0 2
 2 0 0 2 2 0 0 0 2 2 0 2 0 2 0 2 2 0 0 2 2 1 0 0 0 2 0 0 0 0 0 2 0 2 2 0 2
 2 0 0 1 0 0 2 2 0 2 1 2 0 2 0 2 2 2 4 2 1 0 0 2 0 2 0 0 0 0 0 2 0 1 2 0 2
 2 2 2 2 0 0 2 2 2 0 0 1 0 0 0 0 2 2 0 0 0 0 0 2 0 2 0 1 0 0 2 0 0 0 1 0 2
 2 1 0 2 0 0 0 0

### Otras predicciones con otro tipo de valores

In [54]:
# Hacemos la pruebe de prediccion de los clasificadores
print(X_norm.shape)

pred_reg2 = predctOneVSAll(all_theta, X_norm)

print('Precisión del conjunto de entrenamiento  es: {:.2f}%'.format(np.mean(pred_reg2 == y_train.values) * 100))

XPrueba2 = X_norm.iloc[20:700, :].copy()
print(XPrueba2.shape)

XPrueba2 = np.concatenate([np.ones((680, 1)), XPrueba2], axis=1)
print(XPrueba2.shape)

p = np.argmax(sigmoid(XPrueba2.dot(all_theta.T)), axis=1)

print(p)

# print(y_train.iloc[10:200].values)
y_train_array = y_train[20:700].values

print(y_train_array)

(25694, 14)
Precisión del conjunto de entrenamiento  es: 71.99%
(680, 14)
(680, 15)
[2 0 2 0 2 0 2 0 2 0 2 0 2 1 0 1 0 0 2 0 0 2 0 1 0 0 2 2 0 0 0 1 1 2 2 0 0
 0 0 1 2 0 0 2 0 0 0 4 0 0 2 0 2 1 2 0 0 0 0 0 0 0 0 0 2 2 0 4 0 2 2 0 0 0
 0 0 0 2 2 0 2 2 2 0 2 2 1 0 0 0 0 2 2 2 2 0 2 0 0 0 0 0 2 0 0 2 0 0 0 2 0
 2 2 2 0 0 2 2 2 0 0 0 1 0 2 2 0 0 2 2 0 0 0 0 2 0 1 2 0 0 2 0 1 0 2 0 0 0
 0 0 0 0 2 0 0 0 0 0 0 0 0 2 2 2 0 0 2 2 0 2 0 2 2 0 1 2 0 2 1 1 0 2 2 0 0
 0 2 0 0 2 0 2 1 2 0 0 2 4 2 2 0 0 0 0 0 2 2 2 0 2 2 2 2 0 2 1 2 2 0 2 2 1
 2 2 0 0 0 0 0 2 0 2 0 2 2 2 2 2 2 2 0 0 0 2 0 2 2 2 2 0 0 2 2 2 2 2 0 0 2
 0 2 0 2 0 2 2 2 2 0 0 0 0 0 2 0 0 2 1 0 2 1 0 2 0 0 2 1 1 0 0 2 2 2 1 0 0
 0 0 0 2 2 0 0 0 0 0 0 2 0 0 0 0 2 0 1 2 1 2 2 0 2 0 2 2 0 0 2 2 0 0 0 2 2
 0 2 0 2 0 2 2 0 0 2 2 1 0 0 0 2 0 0 0 0 0 2 0 2 2 0 2 2 0 0 1 0 0 2 2 0 2
 1 2 0 2 0 2 2 2 4 2 1 0 0 2 0 2 0 0 0 0 0 2 0 1 2 0 2 2 2 2 2 0 0 2 2 2 0
 0 1 0 0 0 0 2 2 0 0 0 0 0 2 0 2 0 1 0 0 2 0 0 0 1 0 2 2 1 0 2 0 0 0 0 2 1
 2 2 0 0 0 1 0 2

## 1.4.2Validaciones y Prueba

Para las validaciones correspondientes utilizamos datos del entrenamiento dodne aplicamos la forma que 80% y el 20% para la fase de prueba.''

In [55]:
# Aplicaremos datos del dataframe X_test que son el 20% para la prueba 

# Normalizamos el dataset de X_test
X_test_norm = (X_test - mu) / sigma
m_test = len(X_test)
X_test_norm

,age,workclass,fnlwgt,education,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,0.760545,-1.973228,0.414084,-0.597738,0.355351,-0.845199,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
1,2.587333,1.463509,-0.766824,1.214815,-0.036478,-0.609204,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-2.962557,0.283824,-0.565754
2,-0.554742,-1.285880,0.093121,-0.338802,1.139008,1.042765,-0.892912,-1.993562,0.698106,-0.146356,-0.216102,0.778960,0.283824,1.767551
3,-0.043242,2.150856,2.603057,1.214815,-0.036478,-0.609204,2.219030,-1.993562,-1.432448,-0.146356,-0.216102,-0.034413,0.153783,-0.565754
4,0.102901,0.088814,0.126502,0.179070,-0.428306,1.278761,2.219030,0.388385,-1.432448,-0.146356,-0.216102,-0.034413,0.283824,1.767551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6417,0.249044,0.088814,-0.680026,-1.115610,-1.995620,1.750752,-0.892912,0.388385,0.698106,-0.146356,-0.216102,-0.034413,0.283824,-0.565754
6418,-0.043242,-2.660575,-0.782276,-1.115610,-1.995620,-1.553186,0.974253,0.388385,-1.432448,-0.146356,-0.216102,-2.311859,0.283824,-0.565754
6419,1.564332,0.088814,-0.444622,0.179070,-0.428306,-0.845199,-0.892912,0.388385,0.698106,-0.146356,-0.216102,0.778960,0.283824,-0.565754
6420,0.833617,0.088814,-0.483396,1.214815,-0.036478,-0.845199,-0.270524,0.388385,0.698106,0.471444,-0.216102,-1.498485,0.283824,-0.565754


Realizamos el calculo de la Y predicha con los datros de prueba de X_test

In [56]:
# Declaramos un vector donde se calcularan las y_predicha
y_predicha = []

# Calculamos la Y predicha de cada fila de X_test_ready
for i in range(len(X_test_norm)):
    y_predicha.append(predctOneVSAll(all_theta, X_test_norm.iloc[i, :].values.reshape(1, -1)))
    
# Mostramos la Y predicha
y_predicha = np.array(y_predicha).flatten()
y_predicha

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [57]:
# Calculamos su precisión
precision = np.mean(y_predicha == y_test.values) * 100
print("La precisión del conjunto de prueba es de: {:.2f}%".format(precision))

La precisión del conjunto de prueba es de: 71.61%


#### Predicción con los datos de prueba X_test para las predicciones

In [58]:
pred_test = predctOneVSAll(all_theta, X_test_norm)

# Mostramos los resultados predecidos 
pred_test

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

In [59]:
# Mostramos una tabla con los datos de prueba y los resultados predecidos

y_pred_test = pd.DataFrame(pred_test, columns=['marital-status-(pred)'])

tabla_predicciones = pd.concat([X_test, y_test, y_pred_test], axis=1)

# Mostramos la tabla de resultados 
tabla_predicciones.head(20)

,age,workclass,fnlwgt,education,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class,marital-status,marital-status-(pred)
0,49,1,233059,8,11,3,0,4,1,0,0,40,39,0,0,0
1,74,6,109101,15,10,4,0,4,1,0,0,4,39,0,0,0
2,31,2,199368,9,13,11,0,2,1,0,0,50,39,1,0,0
3,38,7,462832,15,10,4,5,2,0,0,0,40,38,0,0,2
4,40,4,202872,11,9,12,5,4,0,0,0,40,39,1,0,0
5,60,0,134152,6,5,0,1,2,1,0,0,35,39,0,1,0
6,43,4,221172,9,13,4,1,4,1,0,0,35,39,0,1,0
7,27,4,59068,11,9,6,0,4,1,0,0,40,39,0,0,0
8,53,5,220786,15,10,12,1,4,0,0,0,60,39,0,4,1
9,45,4,156117,8,11,10,1,4,0,0,0,32,39,0,1,0


### Calculo de la precisión que tiene el modelo de la prueba

In [60]:
# Evaluación del rendimiento de datos de prueba 
accuracy_test = np.mean(pred_test == y_test.values) * 100

print("La precisión del conjunto de prueba es de: {:.2f}%".format(accuracy_test))

La precisión del conjunto de prueba es de: 71.61%


## ================================================================================================================

# Informe sobre la Aplicación de Regularización

## Introducción

En este informe se presenta un análisis comparativo del rendimiento de la regresión logistica con y sin regularización para el conjunto de datos "adult.csv". Se comparan los resultados en tiempo de entrenamiento y convergencia del modelo.


En la regularizacion se obuvo que al hacer las pruebas el valor de la precisión fueron por iguales que al primero, 
una presicion a Precición de entrenamiento: 73.15 % , donde los valores de thetas fueron iguales que al modelo L2, Además no hubo sobreajuste en los datos de entrenamiento. 


## ¿ Cúal fue el resultado o el mejor?

R.- Obteniendo los resultados en esta parte fue a lo contrario porque tanto para la precisión de entrenamiento como para la de prueba fueron lo mismo y las predicciones fuenro las mismas que cuando apliqué sin regularizacion. En este caso ninguna fue mejor porque se obtuvieron los mismos resultados.